In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sns as sns
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, \
    classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

PROBLEM: ŞEKER HASTALIĞI TAHMINI

In [ ]:
df = pd.read_csv("data/diabetes.csv")
df.head()

LOGICTIC REGRESSION

In [ ]:
df["Outcome"].value_counts()

In [ ]:
df.describe().T

In [ ]:
y = df["Outcome"]
x = df.drop(["Outcome"], axis=1)

In [ ]:
y.head()

In [ ]:
x.head()

In [ ]:
log_model = LogisticRegression(solver="liblinear").fit(x, y)
log_model.intercept_

In [ ]:
log_model.coef_

In [ ]:
y_pred = log_model.predict(x)

In [ ]:
confusion_matrix(y, y_pred)

In [ ]:
accuracy_score(y, y_pred)

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
log_model.predict(x)[0:10]

In [ ]:
log_model.predict_proba(x)[0:10]  # sınıf olasılıkları

In [ ]:
logit_roc_auc = roc_auc_score(y, log_model.predict(x))
fpr, tpr, thresholds = roc_curve(y, log_model.predict_proba(x)[:, 1])
plt.figure()
plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("Receiver operating characterisitc")
plt.legend(loc="lower right")
plt.savefig("Log_ROC")
plt.show()

MODEL TUNING

In [ ]:
# Model doğrulama yapılacak

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
log_model = LogisticRegression(solver="liblinear").fit(x_train, y_train)

In [ ]:
y_pred = log_model.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
cross_val_score(log_model, x_test, y_test, cv=10)

KNN

In [ ]:
knn_model = KNeighborsClassifier().fit(x_train, y_train)
knn_model

In [ ]:
y_pred = knn_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
knn_params = {"n_neighbors": np.arange(1, 50)}
knn_cv_model = GridSearchCV(knn_model, knn_params, cv=10, n_jobs=-1, verbose=2).fit(x_train, y_train)

In [ ]:
knn_cv_model.best_score_

In [ ]:
knn_tuned = KNeighborsClassifier(n_neighbors=knn_cv_model.best_params_["n_neighbors"]).fit(x_train, y_train)
y_pred = knn_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
knn_tuned.score(x_test, y_test)

SVM
Amaç: İki sınıfın arasındaki mesafeyi(marjin) maksimum yapmak
Doğrusal form etkili olmayacağı durumlarda boyut artırma yapılır, Kernel hilesi

In [ ]:
svm_model = SVC(kernel="linear").fit(x_train, y_train)

In [ ]:
y_pred = svm_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
svm = SVC()

In [ ]:
svm_params = {"C": np.arange(1, 3), "kernel": ["linear", "rbf"]}
sv_cv_model = GridSearchCV(svm, svm_params, cv=5, n_jobs=-1, verbose=2).fit(x_train, y_train)

In [ ]:
sv_cv_model.best_score_

In [ ]:
svm_tuned = SVC(C=sv_cv_model.best_params_["C"], kernel="linear").fit(x_train, y_train)
y_pred = svm_tuned.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

ANN(ÇOK KATMANLI ALGILAYICILAR)
En küçük hata ile tahmin yapacak katsayıları bulmak

In [ ]:
mlpc_model = MLPClassifier().fit(x_train, y_train)  # doğrusal problem için relu, sınıflandırma için logistic fonsk.

In [ ]:
y_pred = mlpc_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
mlpc_params = {"alpha": [1, 5, 0.1, 0.01, 0.03, 0.005, 0.0001],
               "hidden_layer_sizes": [(10, 10), (100, 100, 100), (100, 100), (3, 5)]}

In [ ]:
mlpc = MLPClassifier(solver="lbfgs", activation="logistic")

In [ ]:
mlp_cv_model = GridSearchCV(mlpc, mlpc_params, cv=10, verbose=2, n_jobs=-1).fit(x_train, y_train)

In [ ]:
mlpc_tuned = MLPClassifier(alpha=5, hidden_layer_sizes=(100, 100)).fit(x_train,
                                                                       y_train)

In [ ]:
y_pred = mlpc_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

CART

In [ ]:
cart_model = DecisionTreeClassifier().fit(x_train, y_train)
y_pred = cart_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
cart = DecisionTreeClassifier()
cart_params = {"depth": [1, 3, 5, 8, 10],
               "min_samples_split": [2, 3, 5, 10, 20, 50]}
cart_cv_model = GridSearchCV(cart, cart_params, cv=10).fit(x_train, y_train)

In [ ]:
cart_tuned = DecisionTreeClassifier(max_depth=5, min_samples_split=20).fit(x_train, y_train)

In [ ]:
y_pred = cart_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

RANDOM FOREST

In [ ]:
rf_model = RandomForestClassifier().fit(x_train, y_train)

In [ ]:
y_pred = rf_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf_params = {"n_estimators": [100, 200, 500, 1000],
             "max_features": [3, 5, 7, 8],
             "min_samples_split": [2, 5, 10, 20]}

In [ ]:
rf_cv_model = GridSearchCV(rf, rf_params, cv=10, n_jobs=-1, verbose=2).fit(x_train, y_train)

In [ ]:
rf_tuned = RandomForestClassifier(max_features=8, min_samples_split=5, n_estimators=500).fit(x_train, y_train)

In [ ]:
y_pred = rf_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Değişken önem düzeyleri

feature_imp = pd.Series(rf_tuned.feature_importances_,
                        index=x_train.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel("Degisken önem skorları")
plt.ylabel("Degiskenler")
plt.title("Degisken önem düzeyleri")
plt.show()

GBM
Modeller serisi kurulur

In [ ]:
gbm_model = GradientBoostingClassifier().fit(x_train, y_train)

In [ ]:
y_pred = gbm_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
gbm = GradientBoostingClassifier()

In [ ]:
gbm_params = {"learning_rate": [0.1, 0.01, 0.001, 0.05],
              "n_estimators": [100, 300, 500, 1000],
              "max_depth": [2, 3, 5, 8]}

In [ ]:
gbm_cv_model = GridSearchCV(gbm, gbm_params, cv=10).fit(x_train, y_train)

In [ ]:
gbm_tuned = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=500).fit(x_train, y_train)

In [ ]:
y_pred = gbm_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Değişken önem düzeyleri

feature_imp = pd.Series(gbm_tuned.feature_importances_,
                        index=x_train.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel("Degisken önem skorları")
plt.ylabel("Degiskenler")
plt.title("Degisken önem düzeyleri")
plt.show()

XGBOOST

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier().fit(x_train, y_train)

In [ ]:
y_pred = xgb_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
xgb = XGBClassifier()
xgb_params = {"n_estimators": [100, 500, 1000],
              "subsample": [0.6, 0.8, 1],
              "max_depth": [3, 5, 7],
              "learning_rate": [0.1, 0.001, 0.01]}

In [ ]:
xgb_cv_model = GridSearchCV(xgb, xgb_params, cv=10, n_jobs=-1, verbose=2).fit(x_train, y_train)

In [ ]:
xgb_cv_model.best_params_

In [ ]:
xgb_tuned = XGBClassifier(learning_rate=0.001, max_depth=7, n_estimators=500, subsample=0.6).fit(x_train, y_train)

In [ ]:
y_pred = xgb_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Değişken önem düzeyleri

feature_imp = pd.Series(xgb_tuned.feature_importances_,
                        index=x_train.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel("Degisken önem skorları")
plt.ylabel("Degiskenler")
plt.title("Degisken önem düzeyleri")
plt.show()

LBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier().fit(x_train, y_train)

In [ ]:
y_pred = lgbm_model.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
lgbm = LGBMClassifier()

In [ ]:
lgbm_params = {"learning_rate": [0.001, 0.01, 0.1],
               "n_estimators": [200, 500, 1000],
               "max_depth": [1, 2, 3, 5, 8]}

In [ ]:
lbgm_cv_model = GridSearchCV(lgbm, lgbm_params, cv=10, verbose=2, n_jobs=-1).fit(x_train, y_train)

In [ ]:
lgbm_tuned = LGBMClassifier(learning_rate=lbgm_cv_model.best_params_["learning_rate"],
                            max_depth=lbgm_cv_model.best_params_["max_depth"],
                            n_estimators=lbgm_cv_model.best_params_["n_estimators"]).fit(x_train, y_train)

In [ ]:
y_pred = lgbm_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
# Değişken önem düzeyleri

feature_imp = pd.Series(lgbm_tuned.feature_importances_,
                        index=x_train.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel("Degisken önem skorları")
plt.ylabel("Degiskenler")
plt.title("Degisken önem düzeyleri")
plt.show()